In [4]:
# 提示词工程示例

# 提示词模板，注意JSON结构中的花括号需要双写以避免被format方法误解析为占位符
prompt_template = '''
请从以下客户描述中提取结构化信息，并严格按照JSON格式返回：

客户描述：{user_input}

请返回以下格式的JSON数据：
{{
    "name": "客户姓名",
    "age": 年龄数字,
    "email": "邮箱地址",
    "phone": "电话号码",
    "interests": ["兴趣1", "兴趣2"]
}}

注意：请确保返回的是有效的JSON格式，不要包含任何其他文字。
'''

# 示例输入
user_input = "我叫张三，今年28岁，邮箱是zhangsan@email.com，电话13812345678，喜欢编程和阅读"

# 使用format方法填充用户输入，此时JSON结构中的双花括号会被还原为单花括号
formatted_prompt = prompt_template.format(user_input=user_input)
print("--- Prompt发送给LLM ---")
print(formatted_prompt)


--- Prompt发送给LLM ---

请从以下客户描述中提取结构化信息，并严格按照JSON格式返回：

客户描述：我叫张三，今年28岁，邮箱是zhangsan@email.com，电话13812345678，喜欢编程和阅读

请返回以下格式的JSON数据：
{
    "name": "客户姓名",
    "age": 年龄数字,
    "email": "邮箱地址",
    "phone": "电话号码",
    "interests": ["兴趣1", "兴趣2"]
}

注意：请确保返回的是有效的JSON格式，不要包含任何其他文字。



In [8]:
from openai import OpenAI # 导入OpenAI库
import json # 导入json库用于处理JSON数据
import os # 导入os库用于获取环境变量

# deepseek 官方文档
# https://api-docs.deepseek.com/zh-cn/guides/function_calling

# 从环境变量获取API Key，或者直接使用提供的TOKEN
# 建议实际应用中使用环境变量，这里为了演示直接使用TOKEN
api_key = os.environ.get("DEEPSEEK_API_KEY", "YOUR_DEFAULT_TOKEN") # 更好的实践

# 定义API的base URL
# 根据用户提供的POST示例和重构要求，使用openai库时需要设置base_url
# 注意：这里使用了大神文档提供的测试地址作为base_url
base_url = 'https://api.deepseek.com' 

# 实例化OpenAI客户端
# api_key 参数是必需的，即使base_url指向非官方API，某些库版本可能仍需要此参数
# base_url 参数指定了API的根地址
client = OpenAI(
    api_key=api_key,
    base_url=base_url,
)

model_name = 'deepseek-chat' # 使用支持 function calling 的模型

# 定义要提供给模型的函数工具
# 这个函数描述了我们希望模型提取并结构化的数据格式
# 结构与requests版本相同，因为这是API要求的格式
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_customer_info", # 函数名称
            "description": "从客户描述中提取结构化信息，包括姓名、年龄、邮箱、电话和兴趣。", # 函数描述
            "parameters": { # 函数参数，使用 JSON Schema 格式定义
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "客户的姓名。"
                    },
                    "age": {
                        "type": "integer",
                        "description": "客户的年龄。"
                    },
                    "email": {
                        "type": "string",
                        "description": "客户的邮箱地址。"
                    },
                    "phone": {
                        "type": "string",
                        "description": "客户的电话号码。"
                    },
                    "interests": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "客户的兴趣列表。"
                    }
                },
                "required": ["name", "age", "email", "phone", "interests"] # 标记所有字段为必需
            }
        }
    }
]

# 构建消息列表
# 将用户输入作为消息内容
messages = [
    {"role": "user", "content": user_input} # 注意这里直接使用了原始的 user_input
]

print("--- 正在调用LLM API (使用 openai 库, Function Calling 模式) ---")

try:
    # 调用 chat completions API
    # 使用 client.chat.completions.create 方法
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools, # 添加 tools 字段
        tool_choice="auto", # 告诉模型自动决定是否调用提供的工具
        temperature=0.0 # 设置温度为0，以获得更稳定的输出
    )

    # openai库返回的是一个对象，直接访问其属性
    # 提取模型的回复内容或函数调用信息
    # 检查是否存在 choices 并且不为空
    print("--------------------------------")
    print(response)
    print("--------------------------------")
    if response.choices and len(response.choices) > 0:
        message = response.choices[0].message # 获取 message 对象

        # 检查是否存在 tool_calls
        if message.tool_calls: # tool_calls 是一个列表，如果非空则为True
            # 提取第一个工具调用信息
            tool_call = message.tool_calls[0]
            if tool_call.type == 'function':
                function_call = tool_call.function # 获取 function 对象
                function_name = function_call.name
                function_args_str = function_call.arguments # 参数是JSON字符串

                print("\n--- LLM返回的函数调用 ---")
                print(f"函数名称: {function_name}")
                print(f"参数 (JSON字符串): {function_args_str}")

                # 尝试解析参数字符串为 Python 字典
                try:
                    function_args = json.loads(function_args_str)
                    print("\n--- 解析后的函数参数 ---")
                    print(json.dumps(function_args, indent=2, ensure_ascii=False))
                    ai_response = function_args # 将解析后的参数作为结果

                    # 示例：进一步处理提取的数据
                    print("\n--- 提取的客户信息 ---")
                    print(f"姓名: {ai_response.get('name', 'N/A')}")
                    print(f"年龄: {ai_response.get('age', 'N/A')}")
                    print(f"邮箱: {ai_response.get('email', 'N/A')}")
                    print(f"电话: {ai_response.get('phone', 'N/A')}")
                    print(f"兴趣: {', '.join(ai_response.get('interests', ['N/A']))}")

                except json.JSONDecodeError:
                    print("\n警告: 无法解析函数参数 JSON 字符串。")
                    ai_response = f"函数调用参数解析失败: {function_args_str}"
            else:
                 print("\n警告: 返回的工具调用类型不是 function。")
                 ai_response = "返回的工具调用类型不是 function"

        elif message.content:
            # 如果没有 tool_calls，检查是否有普通文本回复
            ai_response = message.content
            print("\n--- LLM返回的普通文本回复 (未调用函数) ---")
            print(ai_response)
        else:
             ai_response = "无法从响应中提取内容 (message 字段缺失 tool_calls 或 content)"
             print(f"\n警告: {ai_response}")
             # 可以打印完整的响应对象进行调试
             # print("完整的响应数据:", response)

    else:
        ai_response = "无法从响应中提取内容 (choices 字段缺失或为空)"
        print(f"\n警告: {ai_response}")
        # 可以打印完整的响应对象进行调试
        # print("完整的响应数据:", response)


except Exception as e:
    # 捕获openai库可能抛出的异常 (API错误, 连接错误等)
    print(f"调用LLM API时发生错误: {e}")
    ai_response = "调用失败" # 设置一个默认值，防止后续代码出错

# 注意：原始代码中有一个ai_response变量，这里也保留了，但其主要作用是存储最终结果或错误信息。
# 实际提取的数据已经打印出来了。


--- 正在调用LLM API (使用 openai 库, Function Calling 模式) ---
--------------------------------
ChatCompletion(id='398025d0-e740-4aa9-96cf-4ca3b583f0f4', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_8ffb381b-38d0-4546-85d7-88102dd06943', function=Function(arguments='{"name":"张三","age":28,"email":"zhangsan@email.com","phone":"13812345678","interests":["编程","阅读"]}', name='extract_customer_info'), type='function', index=0)]))], created=1748439861, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_8802369eaa_prod0425fp8', usage=CompletionUsage(completion_tokens=50, prompt_tokens=280, total_tokens=330, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=256), prompt_cache_hit_tokens=256, prompt_cache_miss_to